Imports

In [2]:
import pandas as pd

Read in the file with 11 years of data that we created.

In [4]:
df = pd.read_csv(f"data/anw_data-2022-02-18-10:19.csv", index_col=0)
df.head(2)

,index,Name,Fantasy Points,CQ OBS,Time,CF OBS,Time.1,Boot Camp,Stage 1,Time.2,Stage 2,Time.3,Stage 3,Stage 4,Time.4,Year,run
0,0,Sean Bryan,49.5,Complete (LA),236.84,Complete (LA),369.99,NaN,Complete,126.05,Complete,248.3,Ultimate Crazy Cliffhanger,NaN,NaN,10,NaN
1,1,Drew Drechsel,44.0,Complete (MIA),144.74,Stair Hopper (MIA),Qualified,NaN,Complete,96.2,Complete,231.35,Ultimate Crazy Cliffhanger,NaN,NaN,10,NaN


## Analysis

Let's see what we have time to see. 👀

In [ ]:
df.head(2)


,run,Name,Fantasy Points,CQ OBS,Time,CF OBS,Time.1,Boot Camp,Stage 1,Time.2,Stage 2,Time.3,Stage 3,Stage 4,Time.4
0,NaN,Sean Bryan,49.5,Complete (LA),236.84,Complete (LA),369.99,NaN,Complete,126.05,Complete,248.3,Ultimate Crazy Cliffhanger,NaN,NaN
1,NaN,Drew Drechsel,44.0,Complete (MIA),144.74,Stair Hopper (MIA),Qualified,NaN,Complete,96.2,Complete,231.35,Ultimate Crazy Cliffhanger,NaN,NaN


In [ ]:
df_appearances = df["Name"].value_counts().to_frame().reset_index()
df_appearances.columns=['Name', 'Appearances']
df_appearances

,Name,Appearances
0,Ryan Stratis,11
1,David Campbell,11
2,Travis Rosen,10
3,Lorin Ball,9
4,Chris Wilczewski,9
...,...,...
1516,Omar Payton,1
1517,Zac Eddington,1
1518,Spenser Mestel,1
1519,Robert Taylor,1


In [ ]:
fig = px.histogram(df_appearances, x='Appearances')
fig.update_layout(
    title_text='Most competitors make a single appearance',
    xaxis_title_text='Number of years a competitor appeared',
    yaxis_title_text='Count',
    bargap=0.1,
    showlegend=False,
)


#### How many people appeared at least 7 times in the five years?

In [ ]:
df_appearances[df_appearances['Appearances'] >= 7].count()['Appearances']

22

#### Who are the overall fantasy point leaders?

In [ ]:
df_totals = (
    df.groupby(["Name"])
    .sum()
    .sort_values(by="Fantasy Points", ascending=False)[["Fantasy Points"]]
)
df_totals


,Fantasy Points
Name,
Drew Drechsel,353.0
Joe Moravsky,299.5
Ryan Stratis,289.5
David Campbell,289.0
Travis Rosen,287.0
...,...
Dennis Ruelas,0.0
Charlie Escue,0.0
Tony Geronimo,0.0


In [ ]:
fig2 = px.histogram(df_totals, title='Distribution of yearly fantasy points')
fig2.update_layout(
    bargap=0.1,
    showlegend=False,
    xaxis_title_text='Fantasy Points',
    yaxis_title_text='Count',
)


What are those fantasy points anyway? According to the [website](http://www.anwfantasy.com/how-to-play/), points are scored as follows.

    1 point for every obstacle cleared!

    2 points for City Qualifier course clear
    4 points for City Finals course clears
    4 points for Midoriyama/Las Vegas Stage 1 clears
    6 points for Midoriyama/Las Vegas Stage 2 clears
    8 points for Midoriyama/Las Vegas Stage 3 clears
    10 points for Total Victory
    Additonally a 0.5 point bonus is awarded to the fastest runner for each timed stage

    For example Geoff Britten's "Perfect Season" is scored:
    • 2pts for City Qualifier cleared + 6 obstacles cleared +
    • 4pts for City Finals cleared + 10 obstacles cleared (+0.5 fastest bonus) +
    • 4pts for Stage 1 cleared + 8 obstacles cleared +
    • 6pts for Stage 2 cleared + 6 obstacles cleared (+0.5 fastest bonus) +
    • 8pts for Stage 3 cleared + 8 obstacles cleared +
    • 10pts for Stage 4 cleared + 1 obstacle cleared =
    74 total points!

#### Let's see who had the most fantasy points in a given year.

In [ ]:
fig2 = px.histogram(df_totals, title='Distribution of yearly fantasy points')
fig2.update_layout(
    bargap=0.1,
    showlegend=False,
    xaxis_title_text='Fantasy Points',
    yaxis_title_text='Count',
)


Who and when were the most fantasy points in a season?

In [5]:
df['Fantasy Points'].nlargest()

2135    71.5
856     63.0
2136    63.0
857     56.0
1472    54.0
Name: Fantasy Points, dtype: float64

In [9]:
df.loc[df['Fantasy Points'].nlargest().index][['Name', 'Fantasy Points', 'Year']]

,Name,Fantasy Points,Year
2135,Drew Drechsel,71.5,11
856,Geoff Britten,63.0,7
2136,Daniel Gil,63.0,11
857,Isaac Caldiero,56.0,7
1472,Brian Arnold,54.0,5


Let's plot the top 10 cumulative fantasy point leaders.

In [ ]:
fig3 = px.bar(
    df_fantasy.head(10),
    x="Name",
    y="Fantasy Points",
    color="Name",
    title="Fantasy point leaders (years 1-11)"
)
fig3.update_layout(
    showlegend=False,
)


What's the relationship between ****

Have average fantasy points been increasing?

Has the average number of participants been increasing?